In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import calendar

#Function to get end of the month date for ONI dataset
def get_end_of_month(year, month):
    # Get the last day of the month
    last_day = calendar.monthrange(year, month)[1]
    # Return the date as a string in the format YYYY-MM-DD
    return f"{year}-{month:02d}-{last_day:02d}"

# Function to check if a value is non-numeric
def is_non_numeric(value):
    return not pd.api.types.is_numeric_dtype(type(value))

#Function to bin data to monthly
def resample(group):
    return group.resample('ME').mean()

#Function to bin data to yearly
def resample_year(group):
    return group.resample('YE').mean()

# Get the current working directory 
current_directory = os.getcwd() 

# Print the current working directory 
print(current_directory)

# Define the directory containing the files 
path = current_directory+"\\"
print(path)

filename = path + 'Daily_Data_All_Yearly.csv'

results_df = pd.read_csv(filename) #read csv data into a dataframe

print(results_df)

print(results_df.columns)

# Count non-numeric values in the average (mean) wind speed column
non_numeric_count = results_df['WDSP'].apply(is_non_numeric).sum()

print(f"Number of non-numeric values: {non_numeric_count}")

# Count non-numeric values in the snow depth column
non_numeric_count = results_df['SNDP'].apply(is_non_numeric).sum()

print(f"Number of non-numeric values: {non_numeric_count}")

# Count non-numeric values in the precipitation column
non_numeric_count = results_df['PRCP'].apply(is_non_numeric).sum()

print(f"Number of non-numeric values: {non_numeric_count}")

#There are no non-numeric values

#Find how many missing values are in each column
count = results_df['WDSP'].value_counts().get(999.9, 0)
print(count)
count = results_df['SNDP'].value_counts().get(999.9, 0)
print(count)
count = results_df['slp'].value_counts().get(999.9, 0)
print(count)
count = results_df['SLP'].value_counts().get(999.9, 0)
print(count)
count = results_df['MXSPD'].value_counts().get(999.9, 0)
print(count)
count = results_df['PRCP'].value_counts().get(99.99, 0)
print(count)
count = results_df['GUST'].value_counts().get(999.9, 0)
print(count)
count = results_df['DEWP'].value_counts().get(999.9, 0)
print(count)
count = results_df['STP'].value_counts().get(999.9, 0)
print(count)
count = results_df['MIN'].value_counts().get(9999.9, 0)
print(count)
count = results_df['MAX'].value_counts().get(9999.9, 0)
print(count)
count = results_df['TEMP'].value_counts().get(9999.9, 0)
print(count)

results2_df = results_df.drop(['SLP', 'MXSPD', 'GUST', 'DEWP', 'STP', 'MIN', 'MAX', 'TEMP', 'FRSHTT', 'VISIB', 'MIN_ATTRIBUTES', 'MAX_ATTRIBUTES', 'TEMP_ATTRIBUTES', 'SLP_ATTRIBUTES', 'VISIB_ATTRIBUTES', 'STP_ATTRIBUTES', 'PRCP_ATTRIBUTES', 'DEWP_ATTRIBUTES', 'WDSP_ATTRIBUTES'], axis=1)
results2_df = results2_df.drop(['NAME', 'WMO_ID', 'wind_speed', 'NOAA_file_csv'], axis=1) #wind_speed is from NSRDB data. We are using NOAA variable WDSP instead.
print(results2_df)
# Replace numeric 99.99 values with NaN in PRCP 
results2_df['PRCP'] = results2_df['PRCP'].replace({99.99: np.nan})
# Replace numeric 999.9 values with NaN in WDSP 
results2_df['WDSP'] = results2_df['WDSP'].replace({999.9: np.nan})
# Replace numeric 999.9 values with zero in SNDP 
results2_df['SNDP'] = results2_df['SNDP'].replace({999.9: 0})
print(results2_df)
# Function to check if a value is non-numeric
def is_non_numeric(value):
    return not pd.api.types.is_numeric_dtype(type(value))

# Count non-numeric values in the GHI column
non_numeric_count = results2_df['GHI'].apply(is_non_numeric).sum()

print(f"Number of non-numeric values: {non_numeric_count}")

# Convert the 'time' column from string to datetime
results2_df['date'] = pd.to_datetime(results2_df['time'])

results2_df[results2_df['date'] < datetime.strptime("2024-12-1", "%Y-%m-%d")].isna().sum()

# From binning data from hourly to daily, there are 65 days for which hourly data was missing 
#for all 24 hours. These rows are missing latitude, longitude, and elevation, which we will fill in
#manually in Python using their file_id, so that binning to monthly will not skew location data.

# Find rows with NaN values in column 'latitude'
nan_rows = results2_df.loc[results2_df['latitude'].isnull()]
index = nan_rows.index
print(index)

filename = path + 'filenames_chosen.csv'

locations_df = pd.read_csv(filename) #read location data into a dataframe
print(locations_df)
results3_df = results2_df

#”VLOOKUP” operation to get latitude, longitude, elevation, and region for missing data
results3_df = pd.merge(results3_df, locations_df, left_on='file_id', right_on = 'file_no', how='left')

print(results3_df)

#Confirm location of NaN values by index
print(results3_df.loc[22817:22819])

#Clean data for location information
results3_df['latitude'] = results3_df['latitude_y']
results3_df['longitude'] = results3_df['longitude_y']
results3_df['elevation'] = results3_df['elevation_y']
results3_df = results3_df.drop(['latitude_x', 'longitude_x', 'elevation_x'], axis=1)
results3_df = results3_df.drop(['latitude_y', 'longitude_y', 'elevation_y'], axis=1)
print(results3_df.columns)

results3_df['region'] = results3_df['region_y']
results3_df = results3_df.drop(['region_x', 'region_y', 'filename', 'file_no'], axis=1)
results3_df = results3_df.drop(['missing', 'start_date', 'end_date'], axis=1)
print(results3_df.columns)
#Check location data is now clean
results3_df[results3_df['date'] < datetime.strptime("2025-1-1", "%Y-%m-%d")].isna().sum()

# Show GHI column has no NaN values before 2021
results3_df[results3_df['date'] < datetime.strptime("2021-1-1", "%Y-%m-%d")].isna().sum()
# Replace NaN values with numeric 999999.9 values in GHI for 1-1-2021 until the end of 2024 to bin data to monthly later 
results3_df['GHI'] = results3_df['GHI'].replace({np.nan: 999999.9})

#Confirm NaN values have been removed
results3_df[results3_df['date'] < datetime.strptime("2025-1-1", "%Y-%m-%d")].isna().sum()

#Get WDC-SILSO data into a dataframe

path_SILSO = current_directory+"\\Data\\WDC-SILSO\\"
filename_SILSO = path_SILSO + 'SN_d_tot_V2.0.csv'

solar_activity_df = pd.read_csv(filename_SILSO, sep=';') #read solar activity data into a dataframe

# First three columns are year, month, and day. Fifth column is solar activity (number of sunspots per day)
solar_activity_df.columns = ['Year', 'Month', 'Day', 'Year_Fraction', 'solar_activity', 'error', 'Column7', 'Column8'] 

#Drop last three columns
solar_activity_df = solar_activity_df.iloc[:, :-3]

solar_activity_df['date'] = pd.to_datetime(solar_activity_df[['Year', 'Month', 'Day']])

solar_activity_df = solar_activity_df.drop(['Year_Fraction'], axis=1)

print(solar_activity_df)

combined_df = pd.merge(results3_df, solar_activity_df, on='date', how='left')
print(combined_df)
#Confirm there are no NaN values for solar activity
combined_df[combined_df['date'] < datetime.strptime("2025-1-1", "%Y-%m-%d")].isna().sum()

# Count negative (missing) values
count = (combined_df['solar_activity'] < 0).sum()
print(count)

# Set the date column as the index
combined_df.set_index('date', inplace=True)

#Drop STATION (not needed)
combined_df = combined_df.drop(['STATION'], axis=1)

# Perform linear interpolation to fill NaN values
combined_df['temperatures'] = combined_df['temperatures'].interpolate(method='linear')
combined_df['slp'] = combined_df['slp'].interpolate(method='linear')
combined_df['vapor_pressure'] = combined_df['vapor_pressure'].interpolate(method='linear')
combined_df['wet_bulb_temperature'] = combined_df['wet_bulb_temperature'].interpolate(method='linear')
combined_df['specific_humidity'] = combined_df['specific_humidity'].interpolate(method='linear')
combined_df['WDSP'] = combined_df['WDSP'].interpolate(method='linear')
combined_df['PRCP'] = combined_df['PRCP'].interpolate(method='linear')
combined_df['SNDP'] = combined_df['SNDP'].interpolate(method='linear')

# Reset the index to get back the date column
combined_df.reset_index(inplace=True)

#Confirm there are no NaN values left and data is clean
combined_df[combined_df['date'] < datetime.strptime("2025-1-1", "%Y-%m-%d")].isna().sum()
#Write full daily binned dataset to CSV file
combined_df.to_csv('Final_Daily_Dataset.csv')

# Set 'date' as the index
combined_df.set_index('date', inplace=True)

# Drop DATE and time columns, which can cause problems with binning
combined_df = combined_df.drop(['DATE', 'time'], axis=1)

#Bin the data to monthly
monthly_data = combined_df.groupby('file_id').apply(resample, include_groups=False)

# Reset the index to get back the date column
monthly_data.reset_index(inplace=True)

print(monthly_data)

# Replace back the numeric 999999.9 values with NaN values in GHI 
monthly_data['GHI'] = monthly_data['GHI'].replace({999999.9: np.nan})

print(monthly_data)

# Define the directory containing the files 
path_ONI = current_directory+"\\Data\\ONI\\"
print(path_ONI)

filename_ONI = path_ONI + 'ONIData.txt'

# Read ONI data into a dataframe
ONI_df = pd.read_csv(filename_ONI, delimiter=r'\s+')

# Read ONI data into a dataframe
ONI_df = pd.read_csv(filename_ONI, delimiter=r'\s+')
print(ONI_df)

#create new date column to fill
ONI_df['date'] = '2020-1-1' 

# Set date column to be last day of the month
for i in range(ONI_df.shape[0]):
    ONI_df.loc[i, 'date'] = get_end_of_month(ONI_df['YR'].values[i], ONI_df['MON'].values[i])

#Drop all columns not needed for our final dataset
ONI_df = ONI_df.iloc[:, -2:]

ONI_df.columns = ['ONI', 'date']

print(ONI_df)

monthly_data.reset_index(inplace=True)
ONI_df.reset_index(inplace=True)

ONI_df['date'] = pd.to_datetime(ONI_df['date'])

monthly_data.set_index('date', inplace=True)
ONI_df.set_index('date', inplace=True)

final_monthly_dataset_df = pd.merge(monthly_data, ONI_df, on='date', how='left')

final_monthly_dataset_df.reset_index(inplace=True)

final_monthly_dataset_df = final_monthly_dataset_df.drop(['index_x', 'index_y'], axis=1) #removed 'level_0'

print(final_monthly_dataset_df)

#Confirm there are no NaN values left (except GHI) and data is clean
final_monthly_dataset_df[final_monthly_dataset_df['date'] < datetime.strptime("2025-1-1", "%Y-%m-%d")].isna().sum()

# Count ONI values below -3
count = (final_monthly_dataset_df['ONI'] < -3).sum()
print(count)
# Count ONI values above 3
count = (final_monthly_dataset_df['ONI'] > 3).sum()
print(count)
# Count ONI values less than 3
count = (final_monthly_dataset_df['ONI'] < 3).sum()
print(count)

filename2 = path + 'Final_Monthly_Dataset.csv'

final_monthly_dataset_2006_df = pd.read_csv(filename2) #read csv data into a dataframe
final_monthly_dataset_2006_df = final_monthly_dataset_2006_df.drop(['Unnamed: 0'], axis=1)
final_monthly_dataset_1998_df = final_monthly_dataset_df 

print("1998:")
print(final_monthly_dataset_1998_df)
print("2006:")
print(final_monthly_dataset_2006_df)

final_monthly_dataset_1998_df.set_index('date', inplace=True)

final_monthly_dataset_1998_df.index = pd.to_datetime(final_monthly_dataset_1998_df.index, unit='d')

# Reset the index to get back the date column
final_monthly_dataset_1998_df.reset_index(inplace=True)

#Select only the 24 weather stations we are using
weather_stations = [6678, 6687, 6772, 6783, 6950, 6963, 7239, 7280, 7311, 7321, 7329, 7361, 
                    7414, 7555, 7571, 7585, 7704, 7805, 7816, 7838, 7938, 7979, 7999, 8000]
final_monthly_dataset_2006_df = final_monthly_dataset_2006_df[final_monthly_dataset_2006_df['file_id'].isin(weather_stations)]

final_monthly_dataset_2006_df.set_index('date', inplace=True)

final_monthly_dataset_2006_df.index = pd.to_datetime(final_monthly_dataset_2006_df.index)

# Reset the index to get back the date column
final_monthly_dataset_2006_df.reset_index(inplace=True)

yearly_dataset_df = pd.concat([final_monthly_dataset_1998_df, final_monthly_dataset_2006_df]).sort_values(by=['file_id', 'date'])

# Set 'date' as the index
yearly_dataset_df.set_index('date', inplace=True)

# Drop DATE and time columns, which can cause problems with binning
#yearly_dataset_df = yearly_dataset_df.drop(['DATE', 'time'], axis=1) #removed code since wasn't needed

#Bin the data to monthly
Final_Yearly_Dataset_df = yearly_dataset_df.groupby('file_id').apply(resample_year, include_groups=False)

# Reset the index to get back the date column
Final_Yearly_Dataset_df.reset_index(inplace=True)

Final_Yearly_Dataset_df = Final_Yearly_Dataset_df.drop(['Month', 'Day'], axis=1)

print(Final_Yearly_Dataset_df)

#Write full monthly binned dataset to CSV file
Final_Yearly_Dataset_df.to_csv('Final_Yearly_Dataset.csv')


C:\Users\User
C:\Users\User\
             time  longitude  latitude  elevation  temperatures          slp  \
0        1/1/1998    -86.467    30.393        5.4      6.562500  1034.810416   
1        1/2/1998    -86.467    30.393        5.4      7.958333  1032.070833   
2        1/3/1998    -86.467    30.393        5.4     13.312500  1027.208332   
3        1/4/1998    -86.467    30.393        5.4     15.666667  1023.437499   
4        1/5/1998    -86.467    30.393        5.4     17.833333  1019.037500   
...           ...        ...       ...        ...           ...          ...   
70123  12/27/2005   -119.261    45.826      195.1      4.250000  1014.358333   
70124  12/28/2005   -119.261    45.826      195.1      5.500000  1009.543749   
70125  12/29/2005   -119.261    45.826      195.1      6.291667  1016.537500   
70126  12/30/2005   -119.261    45.826      195.1      6.375000  1002.391667   
70127  12/31/2005   -119.261    45.826      195.1      4.739130   991.554348   

       vap